In [1]:
import sqlite3

# Create a database or connect to one
conn = sqlite3.connect('contacts.db')
c = conn.cursor()

# Create a table
c.execute("""CREATE TABLE IF NOT EXISTS contacts (
            name TEXT,
            phone_number TEXT,
            email TEXT,
            address TEXT
            )""")

conn.commit()
conn.close()


In [2]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

# Function to add a new contact
def add_contact():
    def save_contact():
        name = name_entry.get()
        phone = phone_entry.get()
        email = email_entry.get()
        address = address_entry.get()

        conn = sqlite3.connect('contacts.db')
        c = conn.cursor()
        c.execute("INSERT INTO contacts VALUES (:name, :phone_number, :email, :address)",
                  {'name': name, 'phone_number': phone, 'email': email, 'address': address})
        conn.commit()
        conn.close()
        add_window.destroy()
        view_contacts()

    add_window = tk.Toplevel(root)
    add_window.title("Add New Contact")

    tk.Label(add_window, text="Name").grid(row=0, column=0)
    name_entry = tk.Entry(add_window)
    name_entry.grid(row=0, column=1)

    tk.Label(add_window, text="Phone Number").grid(row=1, column=0)
    phone_entry = tk.Entry(add_window)
    phone_entry.grid(row=1, column=1)

    tk.Label(add_window, text="Email").grid(row=2, column=0)
    email_entry = tk.Entry(add_window)
    email_entry.grid(row=2, column=1)

    tk.Label(add_window, text="Address").grid(row=3, column=0)
    address_entry = tk.Entry(add_window)
    address_entry.grid(row=3, column=1)

    tk.Button(add_window, text="Save Contact", command=save_contact).grid(row=4, column=0, columnspan=2)

# Function to view contacts
def view_contacts():
    for widget in contact_list_frame.winfo_children():
        widget.destroy()

    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute("SELECT * FROM contacts")
    contacts = c.fetchall()
    conn.close()

    for idx, contact in enumerate(contacts):
        tk.Label(contact_list_frame, text=f"{contact[0]}: {contact[1]}").grid(row=idx, column=0)
        tk.Button(contact_list_frame, text="Update", command=lambda c=contact: update_contact(c)).grid(row=idx, column=1)
        tk.Button(contact_list_frame, text="Delete", command=lambda c=contact: delete_contact(c)).grid(row=idx, column=2)

# Function to update a contact
def update_contact(contact):
    def save_updated_contact():
        name = name_entry.get()
        phone = phone_entry.get()
        email = email_entry.get()
        address = address_entry.get()

        conn = sqlite3.connect('contacts.db')
        c = conn.cursor()
        c.execute("""UPDATE contacts SET 
                     name = :name, 
                     phone_number = :phone_number, 
                     email = :email, 
                     address = :address 
                     WHERE name = :old_name AND phone_number = :old_phone_number""",
                  {'name': name, 'phone_number': phone, 'email': email, 'address': address,
                   'old_name': contact[0], 'old_phone_number': contact[1]})
        conn.commit()
        conn.close()
        update_window.destroy()
        view_contacts()

    update_window = tk.Toplevel(root)
    update_window.title("Update Contact")

    tk.Label(update_window, text="Name").grid(row=0, column=0)
    name_entry = tk.Entry(update_window)
    name_entry.insert(0, contact[0])
    name_entry.grid(row=0, column=1)

    tk.Label(update_window, text="Phone Number").grid(row=1, column=0)
    phone_entry = tk.Entry(update_window)
    phone_entry.insert(0, contact[1])
    phone_entry.grid(row=1, column=1)

    tk.Label(update_window, text="Email").grid(row=2, column=0)
    email_entry = tk.Entry(update_window)
    email_entry.insert(0, contact[2])
    email_entry.grid(row=2, column=1)

    tk.Label(update_window, text="Address").grid(row=3, column=0)
    address_entry = tk.Entry(update_window)
    address_entry.insert(0, contact[3])
    address_entry.grid(row=3, column=1)

    tk.Button(update_window, text="Save Changes", command=save_updated_contact).grid(row=4, column=0, columnspan=2)

# Function to delete a contact
def delete_contact(contact):
    response = messagebox.askyesno("Delete Contact", f"Are you sure you want to delete {contact[0]}?")
    if response:
        conn = sqlite3.connect('contacts.db')
        c = conn.cursor()
        c.execute("DELETE FROM contacts WHERE name = :name AND phone_number = :phone_number",
                  {'name': contact[0], 'phone_number': contact[1]})
        conn.commit()
        conn.close()
        view_contacts()

# Function to search for a contact
def search_contact():
    search_window = tk.Toplevel(root)
    search_window.title("Search Contact")

    def perform_search():
        query = search_entry.get()
        conn = sqlite3.connect('contacts.db')
        c = conn.cursor()
        c.execute("SELECT * FROM contacts WHERE name LIKE ? OR phone_number LIKE ?", (f"%{query}%", f"%{query}%"))
        results = c.fetchall()
        conn.close()

        for widget in search_results_frame.winfo_children():
            widget.destroy()

        for idx, contact in enumerate(results):
            tk.Label(search_results_frame, text=f"{contact[0]}: {contact[1]}").grid(row=idx, column=0)

    tk.Label(search_window, text="Search").grid(row=0, column=0)
    search_entry = tk.Entry(search_window)
    search_entry.grid(row=0, column=1)
    tk.Button(search_window, text="Search", command=perform_search).grid(row=0, column=2)

    search_results_frame = tk.Frame(search_window)
    search_results_frame.grid(row=1, column=0, columnspan=3)

# Main application window
root = tk.Tk()
root.title("Contact Management App")

# Add buttons to the main window
tk.Button(root, text="Add Contact", command=add_contact).pack()
tk.Button(root, text="View Contacts", command=view_contacts).pack()
tk.Button(root, text="Search Contact", command=search_contact).pack()

# Frame to display the contact list
contact_list_frame = tk.Frame(root)
contact_list_frame.pack()

view_contacts()

root.mainloop()


2024-08-19 19:08:58.906 python[64075:5277850] Warning: Expected min height of view: (<NSButton: 0x7fe2a8f9c5c0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-08-19 19:08:58.906 python[64075:5277850] Warning: Expected min height of view: (<NSButton: 0x7fe29b18c420>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
